This is a Jupyter notebook created for binary classification model generation.
- We will be loading the data in this notebook from SQL database.
- Perform EDA
- Split data into training and testing datasets
- Perform model training on multiple models
- Select best performing model
- Final evaluation on the model